In [2]:
import numpy as np

In [3]:
import pandas as pd

In [162]:
df=pd.read_csv('5_a.csv')


In [163]:
df.shape

(10100, 2)

In [164]:
df['y'].value_counts()

1.0    10000
0.0      100
Name: y, dtype: int64

In [165]:
df['proba'][0]

0.6373866237658206

#### Function to modify the probabilities

In [166]:
def Change_prob(df):
    k=0
    for i in df['proba']:
        if (i > 0.5):
            df['proba'][k]=1.0
            k=k+1
        else :
            df['proba'][k]=0.0
            k=k+1
    df.rename(columns={'proba':'y_pred'},inplace=True)
    return df

In [167]:
Change_prob(df)

,y,y_pred
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,1.0
6,1.0,1.0
7,1.0,1.0
8,1.0,1.0
9,1.0,1.0


In [168]:
df['y_pred'].value_counts()

1.0    10100
Name: y_pred, dtype: int64

### Task  1,2,4 in a single function  Because the same can be repeated in the 2nd question

In [169]:
def confusion_matrix_Prec_recall_F1_acc(df):
    
    tp,tn,fp,fn=0,0,0,0
    tp = ((df['y']==1.0) & (df['y_pred']==1.0)).sum()
    fp = ((df['y']==0.0) & (df['y_pred']==1.0)).sum()
    fn = ((df['y']==1.0) & (df['y_pred']==0.0)).sum()
    tn = ((df['y']==0.0) & (df['y_pred']==0.0)).sum()
    print('confusion matrix :')
    print(np.array([tp,fp,fn,tn]))
    print('*'*25)
    
    prec=tp/(tp+fp)
    print('prec : {}'.format(prec))
    print('*'*25)
    
    recall=tp/(tp+fn)
    print('recall : {}'.format(recall))
    
    F1_score=2*prec*recall/(prec+recall)
    print('*'*25)
    print('F1_score  : {}'.format(F1_score))
    print('*'*25)
    
    accuracy=(tp+tn)/(tp+tn+fp+fn)
    print('accuracy {} '.format(accuracy))
    return tp,fp,fn,tn,prec,recall,F1_score,accuracy

    

In [170]:
confusion_matrix_Prec_recall_F1_acc(df)

confusion matrix :
[10000   100     0     0]
*************************
prec : 0.9900990099009901
*************************
recall : 1.0
*************************
F1_score  : 0.9950248756218906
*************************
accuracy 0.9900990099009901 


(10000,
 100,
 0,
 0,
 0.9900990099009901,
 1.0,
 0.9950248756218906,
 0.9900990099009901)

### AUC

In [171]:
data=pd.read_csv('5_a.csv')

In [172]:
data.head()

,y,proba
0,1.0,0.637387
1,1.0,0.635165
2,1.0,0.766586
3,1.0,0.724564
4,1.0,0.889199


In [173]:
unique_prob = list(data.proba.unique())

In [174]:
len(unique_prob) # All the values are unique we need to have all treshold values


10100

In [175]:

unique_prob.sort()

In [176]:
# Observation: All the probabilities are greater than 0.5 

In [177]:
from tqdm import tqdm

In [178]:
#Referred this video and followed mapped the example to code this - https://www.youtube.com/watch?v=A_ZKMsZ3f3o
def AUC(data):
    unique_prob = list(data.proba.unique())
    unique_prob.sort(reverse=True)
    tpr=[]
    fpr=[]
    
    for i in unique_prob: # The time complexity is O(n^2)
        y_new=[] # Assigning this here because for every new value of i we need to loop through all the values
        
        for j in data['proba']:
            if (j<i):
                y_new.append(0)
            else :
                y_new.append(1)
        
        
        data['y_new_pred']=y_new # marking this as a feature since we will be using y_new again
        
        tp = (((data['y'])==1) & ((data['y_new_pred'])==1)).sum()
        fp = (((data['y'])==0) & ((data['y_new_pred'])==1)).sum()
        fn = (((data['y'])==1) & ((data['y_new_pred'])==0)).sum()
        tn = (((data['y'])==0) & ((data['y_new_pred'])==0)).sum()
        
        tpr.append(tp/(tp+fn))
        fpr.append(fp/(fp+tn))
        
        
    tpr_plot = sorted(tpr)
    fpr_plot = sorted(fpr)
    AUC = np.trapz(tpr_plot,fpr_plot)

    print('AUC Score:{}'.format(AUC))
    print('tp :{}'.format(tp))
    print('fp :{}'.format(fp))
    print('tn :{}'.format(tn))
    print('fn :{}'.format(fn))
      
    
    
    
    

In [179]:
data=pd.read_csv('5_a.csv')

In [180]:
AUC(data)

AUC Score:0.48829900000000004
tp :10000
fp :100
tn :0
fn :0


# B part

In [181]:
df1=pd.read_csv('5_b.csv')


In [182]:
Change_prob(df1)

,y,y_pred
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [183]:
confusion_matrix_Prec_recall_F1_acc(df1)

confusion matrix :
[  55  239   45 9761]
*************************
prec : 0.1870748299319728
*************************
recall : 0.55
*************************
F1_score  : 0.2791878172588833
*************************
accuracy 0.9718811881188119 


(55,
 239,
 45,
 9761,
 0.1870748299319728,
 0.55,
 0.2791878172588833,
 0.9718811881188119)

In [184]:
data1=pd.read_csv('5_b.csv')

In [185]:
AUC(data1)

AUC Score:0.9377570000000001
tp :100
fp :10000
tn :0
fn :0


# C part

In [186]:
data2=pd.read_csv('5_c.csv')

In [187]:
data2.head()

,y,prob
0,0,0.458521
1,0,0.505037
2,0,0.418652
3,0,0.412057
4,0,0.375579


In [188]:

data2=pd.read_csv('5_c.csv')
unique_prob = list(data2.prob.unique())
unique_prob.sort(reverse=True)

res=[]
for i in unique_prob: # The time complexity is O(n^2)
    y_new=[] # Assigning this here because for every new value of i we need to loop through all the values

    for j in data2['prob']:
        if (j<=i):
            y_new.append(0)
        else :
            y_new.append(1)


    data2['y_new_pred']=y_new # marking this as a feature since we will be using y_new again

    
    fp = (((data2['y'])==0) & ((data2['y_new_pred'])==1)).sum()
    fn = (((data2['y'])==1) & ((data2['y_new_pred'])==0)).sum()
    res.append((500*fn) +(100*fp)) # just a modification to AUC function.
    
    

uniq=unique_prob[res.index(min(res))]

A=min(res)
print('threshold {}'.format(uniq))
print('min value  {}'.format(A))

threshold 0.22987164436159915
min value  141000


# D Task


In [5]:
data3=pd.read_csv('5_d.csv')

In [6]:
data3.head()

,y,pred
0,101.0,100.0
1,120.0,100.0
2,131.0,113.0
3,164.0,125.0
4,154.0,152.0


In [23]:
def MeanSquareErorr(data3):
    length=len(data3)
    mse=0
    for i in range(len(data3)):
        val = (data3['y'][i] - data3['pred'][i]) ** 2
        mse = mse + val           # In every Iteration it loops through each value of y and pred
    mse=1/(length) * mse 
    return mse

In [18]:
(data3['y'][3] - data3['pred'][3])

39.0

In [10]:

def mape(data3):
    length=len(data3)
    numerator=0
    denominator=0
    for i in range(len(data3)):
        numerator+= abs(data3['y'][i] - data3['pred'][i]) 
        denominator+= (data3['y'][i])
    mape = (numerator / denominator) * 100
    return mape

In [11]:
def rsquare(data3):
    length=len(data3)
    ss_res=0
    ss_total=0
    mean=0
    mean =np.mean(data3['y'])
    for i in range(length):
        ss_total += (data3['y'][i] - mean)**2
        ss_res += (data3['y'][i] - data3['pred'][i])**2
    R_square = 1 -(ss_res/ss_total)

    return R_square
    

In [24]:
MeanSquareErorr(data3)

177.16569974554707

In [195]:
mape(data3)

12.91202994009687

In [197]:
rsquare(data3)

0.9563582786990964